# WRDS Data

## Verify installation & Login

https://wrds-www.wharton.upenn.edu/pages/support/programming-wrds/programming-python/jupyterhub-wrds/

In [2]:
try:
    import wrds
except ImportError:
    print("WRDS not installed")


In [3]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

db = wrds.Connection(wrds_username=os.getenv("WRDS_USERNAME")) # ~/.pgpass

Loading library list...
Done


## Check Data Availability

In [4]:
import pandas as pd

In [5]:
price_table_list = [option for option in db.list_tables('optionm') if option.startswith('option')]
price_table_list

['option',
 'option_history',
 'option_price_2002',
 'option_price_2003',
 'option_price_2004',
 'option_price_2005',
 'option_price_2006',
 'option_price_2007',
 'option_price_2008',
 'option_price_2009',
 'option_price_2010',
 'option_price_2011',
 'option_price_2012',
 'option_price_2013',
 'option_price_2014',
 'option_price_2015',
 'option_price_2016',
 'option_price_2017',
 'option_price_2018',
 'option_price_2019',
 'option_price_2020',
 'option_price_2021',
 'option_price_2022',
 'option_price_2023',
 'option_price_view',
 'optionmeurnames',
 'optionmnames']

## Get Data

In [3]:
ticker_name = "SPY"
date = "2023-01-03"


In [7]:
# Get security ID

name_to_secid_df = db.raw_sql("SELECT * FROM optionm.optionmnames WHERE ticker = '"+ticker_name+"'")
name_to_secid_df = name_to_secid_df.dropna(subset=["optionid"])
secid_list = name_to_secid_df["secid"].unique().tolist()
secid_list


[109820.0]

In [8]:
# Get ticker info dataframe on certain date

def get_data_df(secid: float, date: str, label: str):
    """
    label: opprcd, secprd, zerocd, securd, vsurfd, stdopd, distrprojd
    """
    secid = str(secid)
    year = date[:4]
    date = "'"+date+"'"
    
    if label in ["opprcd", "secprd", "vsurfd", "stdopd", "distrprojd"]:
        return db.raw_sql("SELECT * FROM optionm."+label+year+" WHERE secid = "+secid+" AND date = "+date)
    elif label == "zerocd":
        return db.raw_sql("SELECT * FROM optionm."+label+" WHERE date = "+date)
    elif label == "securd":
        return db.raw_sql("SELECT * FROM optionm."+label+" WHERE secid = "+secid)
    else:
        raise ValueError(f"Invalid label: {label}.")


In [10]:
raw_data_path = os.getenv("RAW_DATA_PATH")
date_name = date.translate(str.maketrans('', '', "-"))

# Option price
option_price_dfs = [get_data_df(secid=secid, date=date, label="opprcd") for secid in secid_list]
pd.concat(option_price_dfs, ignore_index=True).to_parquet(raw_data_path+ticker_name+date_name+"_option_price.parquet", 
                                                          engine="pyarrow", 
                                                          compression="snappy")

# Security price
security_price_dfs = [get_data_df(secid=secid, date=date, label="secprd") for secid in secid_list]
pd.concat(security_price_dfs, ignore_index=True).to_parquet(raw_data_path+ticker_name+date_name+"_security_price.parquet", 
                                                            engine="pyarrow", 
                                                            compression="snappy")

# Volatility surface
vk_surface_dfs = [get_data_df(secid=secid, date=date, label="vsurfd") for secid in secid_list]
pd.concat(vk_surface_dfs, ignore_index=True).to_parquet(raw_data_path+ticker_name+date_name+"_volatility_surface.parquet", 
                                                        engine="pyarrow", 
                                                        compression="snappy")

# Standard Option Price
vk_surface_dfs = [get_data_df(secid=secid, date=date, label="stdopd") for secid in secid_list]
pd.concat(vk_surface_dfs, ignore_index=True).to_parquet(raw_data_path+ticker_name+date_name+"_stdoption_price.parquet", 
                                                        engine="pyarrow", 
                                                        compression="snappy")

# Distribution Projection
distr_proj_dfs = [get_data_df(secid=secid, date=date, label="distrprojd") for secid in secid_list]
pd.concat(distr_proj_dfs, ignore_index=True).to_parquet(raw_data_path+ticker_name+date_name+"_distr_proj.parquet", 
                                                        engine="pyarrow", 
                                                        compression="snappy")

# Zero Curve
zero_curve_df = get_data_df(secid=secid_list[0], date=date, label="zerocd")
zero_curve_df.to_parquet(raw_data_path+date_name+"_zero_curve.parquet", 
                         engine="pyarrow",
                         compression="snappy")

# Securd
securd_dfs = [get_data_df(secid=secid, date=date, label="securd") for secid in secid_list]
pd.concat(securd_dfs, ignore_index=True).to_parquet(raw_data_path+ticker_name+"_securd.parquet", 
                                                    engine="pyarrow", 
                                                    compression="snappy")


In [11]:
# Log out

db.close()

## Data Preprocessing

In [4]:
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from datetime import timedelta, datetime

raw_data_path = os.getenv("RAW_DATA_PATH")
date_name = date.translate(str.maketrans('', '', "-"))
date = datetime.strptime(date, '%Y-%m-%d')

In [5]:
op_df = pd.read_parquet(raw_data_path+ticker_name+date_name+"_option_price.parquet")

op_cols = ['date','exdate','secid',
           'optionid','cp_flag','strike_price',
           'best_bid','best_offer','volume',
           'open_interest','impl_volatility']
op_df = op_df[op_cols]

op_df["date"]  = pd.to_datetime(op_df["date"])
op_df["exdate"]= pd.to_datetime(op_df["exdate"])
op_df['mid'] = (op_df['best_bid'] + op_df['best_offer']) / 2.0
op_df = op_df.rename(columns={'strike_price': 'K'})
op_df['T'] = (op_df["exdate"] - op_df["date"]).dt.days / 365.0

op_df.head()


,date,exdate,secid,optionid,cp_flag,K,best_bid,best_offer,volume,open_interest,impl_volatility,mid,T
0,2023-01-03,2023-01-20,109820.0,136011273.0,P,410000.0,28.8,29.06,148.0,6839.0,<NA>,28.93,0.046575
1,2023-01-03,2023-01-20,109820.0,150272659.0,P,411000.0,29.8,30.06,2.0,371.0,<NA>,29.93,0.046575
2,2023-01-03,2023-01-20,109820.0,150272660.0,P,412000.0,30.8,31.06,0.0,456.0,<NA>,30.93,0.046575
3,2023-01-03,2023-01-20,109820.0,150272661.0,P,413000.0,31.8,32.06,1.0,4.0,<NA>,31.93,0.046575
4,2023-01-03,2023-01-20,109820.0,150272662.0,P,414000.0,32.8,33.06,0.0,41.0,<NA>,32.93,0.046575


In [6]:
zc_df = pd.read_parquet(raw_data_path+date_name+"_zero_curve.parquet")

zc_df = zc_df.drop(columns=['date'])
zc_df

,days,rate
0,10.0,4.002951
1,30.0,4.111739
2,60.0,4.26291
3,91.0,4.404409
4,122.0,4.531454
5,152.0,4.641112
6,182.0,4.738162
7,273.0,4.960102
8,365.0,5.083813
9,547.0,5.081657


In [7]:
zc_days = zc_df['days'].to_numpy(dtype=float)
zc_rate = zc_df['rate'].to_numpy(dtype=float)

def r_of_T(T_years: np.ndarray) -> np.ndarray:
    days = np.maximum(T_years, 0) * 365.0
    r = np.interp(days, zc_days, zc_rate, left=zc_rate[0], right=zc_rate[-1])
    return r

def discount_factor(T_years: np.ndarray) -> np.ndarray:
    r = r_of_T(np.asarray(T_years, dtype=float))
    return np.exp(-r * np.asarray(T_years, dtype=float))


In [8]:
dp_df = pd.read_parquet(raw_data_path+ticker_name+date_name+"_distr_proj.parquet")
sc_df = pd.read_parquet(raw_data_path+ticker_name+"_securd.parquet")

dp_df["date"]  = pd.to_datetime(dp_df["date"])
dp_df["exdate"]= pd.to_datetime(dp_df["exdate"])

dp_df.head()

,secid,date,exdate,amount
0,109820.0,2023-01-03,2023-03-17,0.004678
1,109820.0,2023-01-03,2023-06-16,0.004678
2,109820.0,2023-01-03,2023-09-15,0.004678
3,109820.0,2023-01-03,2023-12-15,0.004678
4,109820.0,2023-01-03,2024-03-15,0.004678
